In [11]:
import plotly.express as px
import pandas as pd

# Carica il dataset
df = pd.read_csv('exports-calendar-year.csv')

# Filtra il DataFrame per includere solo i paesi selezionati
selected_countries = ['Brazil', 'Vietnam', 'Colombia', 'Indonesia', 'Ethiopia', 'Honduras']
df = df[df['exports'].isin(selected_countries)]

# Assegna a ogni paese selezionato un indice unico (posizione Y)
df['CountryIndex'] = range(len(df))

# Prepara i dati per il grafico a bolle
df_long = df.melt(id_vars=['exports', 'CountryIndex'], var_name='Year', value_name='ExportVolume')
df_long['Year'] = df_long['Year'].astype(int)

# Crea il grafico a bolle animato
fig = px.scatter(df_long, x='Year', y='CountryIndex', size='ExportVolume', color='exports',
                 hover_name='exports', animation_frame='Year',
                 size_max=60, range_x=[df_long['Year'].min(), df_long['Year'].max()])

# Aggiusta l'asse Y per mostrare i nomi dei paesi
fig.update_yaxes(tickvals=df['CountryIndex'], ticktext=df['exports'])

# Mostra il grafico
fig.show()

In [14]:
import plotly.express as px
import pandas as pd

# Carica il dataset
df = pd.read_csv('exports-calendar-year.csv')

# Filtra il DataFrame per includere solo i paesi selezionati
selected_countries = ['Brazil', 'Vietnam', 'Colombia', 'Indonesia', 'Ethiopia', 'Honduras']
df = df[df['exports'].isin(selected_countries)]

# Assegna a ogni paese selezionato un indice unico (posizione Y)
df['CountryIndex'] = range(len(df))

# Prepara i dati per il grafico a bolle
df_long = df.melt(id_vars=['exports', 'CountryIndex'], var_name='Year', value_name='ExportVolume')
df_long['Year'] = df_long['Year'].astype(int)

# Crea il grafico a bolle animato
fig = px.scatter(df_long, x='Year', y='CountryIndex', size='ExportVolume', color='ExportVolume',
                 hover_name='exports', animation_frame='Year',
                 size_max=60, range_x=[df_long['Year'].min(), df_long['Year'].max()])

# Aggiusta l'asse Y per mostrare i nomi dei paesi
fig.update_yaxes(tickvals=df['CountryIndex'], ticktext=df['exports'])

# Mostra il grafico
fig.show()